In [20]:
import numpy as np
import pandas as pd
from pathlib import Path
import time, sys
from scipy import stats
from zive_util_vu import zive_read_df_rpeaks


def runtime(s):
    hours, remainder = divmod(s, 3600)
    minutes, seconds = divmod(remainder, 60)
    print('Runtime: {:02}:{:02}:{:02}'.format(int(hours), int(minutes), int(seconds)))

def create_SubjCode(userNr, recordingNr):
    """
    Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
    
    zive atveju: SubjCode = userNr + recordingNr, kur userNr >= 1000,
    pvz. SubjCode = 10001
    mit2zive atveju: SubjCode = userNr,  kur userNr < 1000,
    pvz. SubjCode = 101
    Parameters
    ------------
        userNr: int
        recordingNr: int
    Return
    -----------
        SubjCode : int
    """      
    # SubjCode = userNr + recordingNr
    # pvz. SubjCode = 10002
    
    if (userNr < 1000):
        return userNr
    else:        
        str_code = str(userNr) + str(recordingNr)   
        SubjCode = int(str_code)  
        return SubjCode

def get_SubjCode(idx, all_beats_attr):
    row = all_beats_attr.loc[idx]
    SubjCode = create_SubjCode(row['userNr'],  row['recordingNr'])
    return SubjCode

def split_SubjCode(SubjCode):
    """
    Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
    
    zive atveju: SubjCode = int(str(userNr) + str(registrationNr)), kur userNr >= 1000,
    pvz. SubjCode = 10001
    mit2zive atveju: SubjCode = userNr,  kur userNr < 1000,
    pvz. SubjCode = 101
    https://www.adamsmith.haus/python/answers/how-to-get-the-part-of-a-string-before-a-specific-character-in-python
    Parameters
    ------------
        SubjCode: int
    Return
    -----------
        userNr: int
        recordingNr: int
    """   
    if (SubjCode < 1000):
        userNr = SubjCode
        recordingNr = 0   
        return userNr, recordingNr
    else:        
        str_code = str(SubjCode) 
        chars = list(str_code)
        str1 =""
        userNr = int(str1.join(chars[:4]))
        str2 =""
        recordingNr = int(str2.join(chars[4:]))
        return userNr, recordingNr
 
def get_seq_start_end(signal_length,i_sample,window_left_side,window_right_side):
    # Nustatome išskiriamos EKG sekos pradžią ir pabaigą
    seq_start = i_sample - window_left_side
    seq_end = i_sample + window_right_side
    if (seq_start < 0 or seq_end > signal_length):
        # print("\nseq_start: ", seq_start, " seq_end: ", seq_end)
        return (None,None)
    else:    
        return (seq_start, seq_end)

def read_rec(rec_dir, SubjCode):
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    signal = np.load(file_path, mmap_mode='r')
    # print(f"SubjCode: {SubjCode}  signal.shape: {signal.shape}")
    return signal

def read_seq_RR_arr(rec_dir, all_beats_attr, idx, wl_side, wr_side, nl_steps=0, nr_steps=0):
# Nuskaito ir pateikia EKG seką apie R dantelį seq: reikšmiu kiekis wl_side - iš kairės pusės, 
# reikšmiu kiekis wr_side - iš dešinės pusės, R dantelio vietą EKG įraše sample,
# ir atitinkamo pūpsnio klasės numerį label: 0, 1, 2.
# Taip pat pateikia seką RRl_arr iš nl_steps RR reikšmių tarp iš eilės einančių R dantelių į kairę nuo einamo R dantelio 
# ir seką RRr_arr nr_steps RR reikšmių tarp iš eilės einančių R dantelių į dešinę nuo einamo R dantelio dantelio.
# Seka iš kairės RRl_arr prasideda nuo tolimiausio nuo R dantelio atskaitymo ir jai pasibaigus,
# toliau ją pratesia RRl_arr. 

    row = all_beats_attr.loc[idx]
    SubjCode = get_SubjCode(idx, all_beats_attr)

    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    
    signal = np.load(file_path, mmap_mode='r')     
    signal_length = signal.shape[0]
    (seq_start, seq_end)  = get_seq_start_end(signal_length, row['sample'], wl_side, wr_side)

# **************************** Tikrinimai ******************************************************************
    # Tikriname, ar sekos langas neišeina už įrašo ribų
    if (seq_start == None or seq_end == None):
        raise Exception(f"Klaida! {idx}: Sekos lango rėžiai už EKG įrašo {SubjCode} ribų.") 
        # Reikia mažinti wl_side ar wr_side, arba koreguoti idx ribas 
    else:    
        seq = signal[seq_start:seq_end]
        sample = row['sample']
        label = row['label']

    # Tikriname, ar skaičiuodami RR neišeisime už all_beats_attr ribų
    if (idx + nr_steps) >= len(all_beats_attr):
        txt = f"Klaida 1! idx, nl_steps: {idx}, {nr_steps} Skaičiuojant RR viršijama pūpsnių atributo masyvo riba." 
        raise Exception(txt)  
        # Reikia mažinti nr_steps arba koreguoti viršutinę idx ribą
    
    if ((idx - nl_steps) < 0):
        txt = f"Klaida 2! idx, nl_steps: {idx}, {nl_steps} Skaičiuojant RR išeinama už pūpsnių atributo masyvo ribų."
        raise Exception(txt)  
        # Reikia mažinti nl_steps arba didinti apatinę idx ribą

    # Tikriname, ar skaičiuodami RR neišeisime už EKG įrašo SubjCode ribų
    SubjCodeRight = get_SubjCode(idx + nr_steps, all_beats_attr)
    if (SubjCodeRight != SubjCode):
        txt = f"Klaida 3! idx, nl_steps: {idx}, {nr_steps} Skaičiuojant RR viršijama EKG įrašo {SubjCode} viršutinė riba."
        raise Exception(txt)  
        # Reikia mažinti nr_steps arba mažinti max idx 

    SubjCodeLeft = get_SubjCode(idx - nl_steps, all_beats_attr)
    if (SubjCodeLeft != SubjCode):
        txt = f"Klaida 4! idx, nl_steps: {idx}, {nl_steps} Skaičiuojant RR išeinama už EKG įrašo {SubjCode} apatinės ribos."
        raise Exception(txt)  
        # Reikia koreguoti nl_steps arba koreguoti apatinę idx ribas 

# **************************** Tikrinimų pabaiga ******************************************************************

    # Suformuojame RR sekas kairėje ir dešinėje idx atžvilgiu
    if (nl_steps != 0):
        RRl_arr = np.zeros(shape=(nl_steps), dtype=int)
        for i in range(nl_steps):
            RRl_arr[nl_steps-i-1] = all_beats_attr.loc[idx-i, 'sample'] - all_beats_attr.loc[idx-i-1, 'sample']
    else:    
        RRl_arr = None

    if (nr_steps != 0):
        RRr_arr = np.zeros(shape=(nr_steps), dtype=int)
        for i in range(nr_steps):
            RRr_arr[i] = all_beats_attr.loc[idx+i+1, 'sample'] - all_beats_attr.loc[idx+i, 'sample']
    else:
        RRr_arr = None        
    
    return seq, sample, label, RRl_arr, RRr_arr


def create_set(rec_dir, all_beats_attr, ind_lst, window_left_side, window_right_side, nl_steps, nr_steps):
    # Panaudojant sekų atributų freimą, sukuriamas užduoto ilgio sekų ir klasių numerių masyvai, 
    # tinkami klasifikatoriaus mokymui ir tikslumo vertinimui
    
    seq_length = window_left_side + window_right_side

    # Suformuojame masyvus pildymui
    set_len = len(ind_lst)
    X = np.empty((set_len, seq_length))
    y = np.empty((set_len), dtype=int)

    # Pildymas
    count = 0
    for idx in ind_lst:
        seq_1d, sample, label, RRl_arr, RRr_arr = read_seq_RR_arr(rec_dir, all_beats_attr, idx,
                                             window_left_side, window_right_side, nl_steps, nr_steps)
        if (label != None):
            X[count,:] = seq_1d 
            y[count] = label
            count +=1
        else:
            print("Klaida!", idx)    

    # Koreguojame ilgius (šiam variantui bereikalinga, nes jei kai kurie label == None, išduos klaidą)
    X_set = np.resize(X, (count, seq_length))
    y_set = np.resize(y, (count))
    return X_set, y_set


# TESTAVIMAS

def print_stats(stats, arr_values):
    print(f'min: {stats.minmax[0]:.5f}, max: {stats.minmax[1]:.4f}  mean: {stats.mean:.5f}  standard: {np.std(arr_values):.5f}')
    print(f'variance: {stats.variance:.5f}  skewness: {stats.skewness:.5f}  kurtosis: {stats.kurtosis:.5f}')


import warnings
# warnings.filterwarnings("ignore")

my_os= sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'


# Vietinės talpyklos aplankas
db_folder = 'DUOM_VU'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)
rec_dir = Path(db_path,'records_npy')

print("Duomenys skaitomi iš:", rec_dir)
print("\n")

# Testas sekų skaitymui su read_seq_RR
print("\nTestas sekų skaitymui su read_seq_RR")

# *********************** parametrai ********************************
wl_side, wr_side = 5,5
nl_steps, nr_steps = 0,3
idx_min, idx_max = 0,0 
# idx_min, idx_max = 322990,322993 
print(f"wl_side = {wl_side} wr_side = {wl_side}")
print(f"nl_steps = {nl_steps} nr_steps = {nr_steps}")
print(f"idx_min = {idx_min} idx_max = {idx_max}")
print("\n")
# *********************************************************************

idxs = [i for i in range(idx_min,idx_max+1)]

# Nuskaitome pūpsnių atributų failą
file_path = Path(rec_dir, 'all_beats_attr_z.csv')
all_beats_attr = pd.read_csv(file_path, index_col=0)

                    # Testas klaidų tikrinimui
for idx in idxs:
    print("\n")
    seq_1d, sample, label, RRl_arr, RRr_arr = read_seq_RR_arr(rec_dir, all_beats_attr, idx,
     wl_side, wr_side, nl_steps, nr_steps)
    SubjCode = get_SubjCode(idx, all_beats_attr)
    print(f"SubjCode = {SubjCode} idx = {idx}  seq_1d.shape = {seq_1d.shape}  sample = {sample}  label = {label}")
    print(f"RRl_arr: {RRl_arr}")
    print(f"RRr_arr: {RRr_arr}")
    print('seq_1d:', seq_1d)

    # Kontrolinis skaitymas
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()
    print("\natr_sample:")
    print(atr_sample[:10])

    print("\natr_symbol:")
    print(atr_symbol[:10])

    signal = read_rec(rec_dir, SubjCode)
    print("\nSignal: 166")
    print(signal[146:167])
    print(signal[166:186])

    # Statistika
    print("\nStatistics:")
    statistics = stats.describe(seq_1d)
    print_stats(statistics, seq_1d)
    print("\n")

# Suformuojame  imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

# Imties pacientai
DS1 = [10020, 10021, 10051]

# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 3

start_time = time.time()

index_beats_attr = all_beats_attr.index 
ind_lst = []
for SubjCode in DS1:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = index_beats_attr[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    ind_lst.extend(selected_ind.to_list())

print("len(ind_lst):", len(ind_lst))
nl_steps, nr_steps = 3,3

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
X, y = create_set(rec_dir, all_beats_attr, ind_lst, wl_side, wr_side, nl_steps, nr_steps)
print("\nImties dimensijos")
print(f"X.shape: {X.shape} y.shape: {y.shape}")

end_time = time.time()
runtime(end_time-start_time)


OS in my system :  win32
Duomenys skaitomi iš: D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy



Testas sekų skaitymui su read_seq_RR
wl_side = 5 wr_side = 5
nl_steps = 0 nr_steps = 3
idx_min = 0 idx_max = 0




SubjCode = 10000 idx = 0  seq_1d.shape = (10,)  sample = 166  label = 0
RRl_arr: None
RRr_arr: [167 173 175]
seq_1d: [-1.49947561 -1.33649948 -1.05039718 -0.64593709 -0.09331689  0.34775749
  0.20385484 -0.52604661 -1.30805955 -1.66364383]

atr_sample:
[ 166  333  506  681  853 1028 1205 1371 1530 1688]

atr_symbol:
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N']

Signal: 166
[-1.37515735 -1.37260286 -1.37089987 -1.37055927 -1.37004838 -1.37055927
 -1.3666424  -1.35863834 -1.35591356 -1.35693535 -1.35557296 -1.35557296
 -1.36783449 -1.42233016 -1.51258863 -1.49947561 -1.33649948 -1.05039718
 -0.64593709 -0.09331689  0.34775749]
[ 0.34775749  0.20385484 -0.52604661 -1.30805955 -1.66364383 -1.66364383
 -1.54749992 -1.45196219 -1.38861097 -1.35352937 -1.33854306 -1.34109755
 -1.33973516 -1.